In [44]:
import pandas as pd
import numpy as np
import re

sex = 'female'
 
# 读取数据
df = pd.read_excel('data.xlsx',sheet_name='男胎检测数据' if sex == 'male' else '女胎检测数据')


# 去除所有列的空值列
df.dropna(axis=1, how='all', inplace=True)



# 1. 末次月经：转为日期类型
df['末次月经'] = pd.to_datetime(df['末次月经'], errors='coerce')


# 3. 检测孕周：12w+3 → 浮点型孕周
def parse_gestational_age(ga_str):
    if pd.isnull(ga_str):
        return np.nan
    match = re.match(r'(\d+)w\+(\d+)', str(ga_str))
    if match:
        weeks = int(match.group(1))
        days = int(match.group(2))
        return weeks  + days /7
    match = re.match(r'(\d+)w', str(ga_str))
    if match:
        weeks = int(match.group(1))
        return weeks
    else:
        return np.nan

df['检测孕周_周数'] = df['检测孕周'].apply(parse_gestational_age)

# 4. 染色体的非整倍体：保留原文本（可为空）
# 无需处理，缺失值保留

# 5. 怀孕次数：>=3 → 3
def parse_pregnancy_count(val):
    if pd.isnull(val):
        return 3
    if str(val).strip() == '>=3':
        return 3
    try:
        return int(val)
    except:
        return 3

df['怀孕次数'] = df['怀孕次数'].apply(parse_pregnancy_count)

# 6. 胎儿是否健康：是/否 → 1/0
df['胎儿是否健康'] = df['胎儿是否健康'].map({'是': 1, '否': 0})


# 删除末次月经列为空的
df = df.dropna(subset=['末次月经'])


# 末次月经数据是YYYY-MM-DD,直接转为时间戳
df['末次月经'] = df['末次月经'].astype(np.int64) // 10**9


# 检测日期格式是YYYYMMDD，转为时间戳
df['检测日期'] = pd.to_datetime(df['检测日期'], format='%Y%m%d').astype(np.int64) // 10**9


# 将 染色体的非整倍体 列的空值转为0，并保留列
df['染色体的非整倍体'] = df['染色体的非整倍体'].fillna(0)


# IVF妊娠列包含自然和人工，将自然受孕对应为0，人工授精对应为1
df['IVF妊娠'] = df['IVF妊娠'].map({'自然受孕': 0, 'IUI（人工授精）': 1,'IVF（试管婴儿）': 2})


# 去除列名多余的空格
df.rename(columns={c: c.strip() for c in df.columns}, inplace=True)


# 输出清洗后的数据
df.to_csv('data_cleaned.csv', index=False)

print("数据清洗完成，已保存为 data_cleaned.xlsx")


数据清洗完成，已保存为 data_cleaned.xlsx


In [45]:
df['染色体的非整倍体']

0        0
1        0
2        0
3        0
4        0
      ... 
600    T18
601      0
602      0
603      0
604      0
Name: 染色体的非整倍体, Length: 597, dtype: object

还要合并部分...

In [46]:
df = pd.read_csv('data_cleaned.csv')

# 选数值列并生成 agg 字典
num_cols = df.select_dtypes(include='number').columns
agg_dict = {c:'mean' for c in num_cols if c not in ['染色体的非整倍体']}
df_clean = df.groupby(['孕妇代码','检测抽血次数'], as_index=False) \
             .agg({**agg_dict, '染色体的非整倍体':'first'})
df_clean.to_csv('data_cleaned.csv', index=False)

In [47]:
df_clean['染色体的非整倍体']

0        0
1        0
2        0
3        0
4        0
      ... 
541    T18
542      0
543      0
544      0
545      0
Name: 染色体的非整倍体, Length: 546, dtype: object

In [48]:
import seaborn as sns
import matplotlib.pyplot as plt

# 设置中文字体为黑体（SimHei），解决中文乱码
plt.rcParams['font.sans-serif'] = ['SimHei']  
# 解决负号 '-' 显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False

# 1. 读取数据
df = pd.read_csv('data_cleaned.csv')

# 2. 只保留关键列非空（Y染色体浓度）
df_sex = df[df['Y染色体浓度' if sex == 'male' else 'X染色体浓度'].notnull()]

# 3. 只保留数值型列
numeric_df = df_sex.select_dtypes(include=['float64', 'int64'])

numeric_df = numeric_df.drop(['序号'], axis=1)


# 保存各个特征的mean和标准差到feature_mean_std.csv
feature_mean = numeric_df.mean()
feature_std = numeric_df.std()
feature_mean_std = pd.DataFrame({'特征名': numeric_df.columns,
                                 '均值': feature_mean,
                                 '标准差': feature_std})
feature_mean_std.to_csv('feature_mean_std_'+sex+'.csv', index=False)


# 4. 缺失值填充（这里用列的中位数填充，你也可以改成均值或0）
numeric_df_filled = numeric_df.apply(lambda col: col.fillna(col.median()))

excluded_cols = ['孕妇代码','孕妇BMI','IVF妊娠', '怀孕次数', '生产次数', '胎儿是否健康',"Y染色体浓度",'X染色体浓度','染色体的非整倍体']


merged = pd.concat([df_sex['孕妇代码'],df_sex['染色体的非整倍体'], numeric_df_filled], axis=1)


# 5. 3σ准则剔除异常值（按列处理）
def three_sigma_filter(df, 
                       excluded_cols=excluded_cols):
    df_filtered = df.copy()
    for col in df_filtered.columns:
        if col in excluded_cols:
            continue
        mean = df_filtered[col].mean()
        std = df_filtered[col].std()
        mask = (df_filtered[col] >= mean - 3*std) & (df_filtered[col] <= mean + 3*std)
        df_filtered = df_filtered[mask]
    return df_filtered
merged = three_sigma_filter(merged)


merged.to_csv('data_processed_'+sex+'.csv', index=False)


# 标准化numeric_df
# 不标准化部分列
std_cols = [col for col in merged.columns if col not in excluded_cols]

# 标准化部分列
merged[std_cols] = (merged[std_cols] - merged[std_cols].mean()) / merged[std_cols].std()


# 保存为data_processed.csv
merged.to_csv('data_processed_'+sex+'_std.csv', index=False)




In [49]:
# 6. 计算相关系数矩阵（pairwise 方式，自动忽略缺失值）
corr_matrix = numeric_df_filled.corr(method='pearson')

# 7. 提取与 Y染色体浓度 的相关性
target_corr = corr_matrix['Y染色体浓度'].sort_values(key=abs, ascending=False)

# 排序后输出
target_corr = target_corr.sort_values(key=abs, ascending=False)
print("与 Y染色体浓度 的相关性（清洗+填充后）：")
print(target_corr)

# 8. 可视化热力图（前15个相关性最高的变量）
top_vars = target_corr.index[:15]
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix.loc[top_vars, top_vars], annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Top Related Variables with Y染色体浓度 (Cleaned & Filled Data)')
plt.show()

# 9. 根据阈值筛选变量（绝对相关性 > 0.2）
threshold = 0.2
selected_vars = target_corr[abs(target_corr) > threshold].index.tolist()
selected_vars = [v for v in selected_vars if v != 'Y染色体浓度']

print(f"筛选出的自变量（相关性 > {threshold}）：")
print(selected_vars)

KeyError: 'Y染色体浓度'